CSV 파일 인코딩 변환(완성형(확장완성형 CP949) 파일만)


In [47]:
from pathlib import Path

dir_to = "../data/processed/개인통행실태조사(2021년기준)/"
Path(dir_to).mkdir(exist_ok=True)

csvfile_encodings = {
    "../data/raw/개인통행실태조사(2021년기준)/①개인특성.csv": "CP949",
    "../data/raw/개인통행실태조사(2021년기준)/②이동특성.csv": "CP949",
}

csvfiles = []

for file, encoding in csvfile_encodings.items():
    csvfile_to = (Path(dir_to) / Path(file).name).as_posix()
    csvfiles.append(csvfile_to)

    # bash에서 iconv 이용해 인코딩을 utf-8로 변환
    command = f"iconv -f {encoding} -t UTF-8 '{Path(file).as_posix()}' -o '{Path(csvfile_to).as_posix()}'"
    !bash -c "{command}"

print(csvfiles)

['../data/processed/개인통행실태조사(2021년기준)/①개인특성.csv', '../data/processed/개인통행실태조사(2021년기준)/②이동특성.csv']


CSV, 엑셀 파일 로드


In [92]:
from pathlib import Path
import pandas as pd
import duckdb

csvfiles = [
    "../data/processed/개인통행실태조사(2021년기준)/①개인특성.csv",
    "../data/processed/개인통행실태조사(2021년기준)/②이동특성.csv",
]

# Connect to DuckDB
con = duckdb.connect(database="../data/개인통행실태조사.db")

# Loop through the CSV files
for file_path in csvfiles:
    # Create a unique table name and replace spaces and parentheses with underscores
    table_name = Path(file_path).stem
    table_name = table_name.replace(" ", "_").replace("(", "_").replace(")", "_")

    # Forcefully create the table by dropping it if it exists first
    con.sql(f"DROP TABLE IF EXISTS {table_name}")
    con.sql(f"CREATE TABLE {table_name} AS SELECT * FROM read_csv('{file_path}')")

# Example: Query a table in DuckDB
result = con.sql("SELECT * FROM ①개인특성 USING SAMPLE 5").df()
display(result)

# Close the DuckDB connection when done
con.close()


,idx,SQ1_4,SQ1_5,SQ1_6,SQ1_7,SQ1_8,SQ2,SQ3,SQ3_1,Q1_1,...,D1,D1_2,D1_2_a,D1_3,D1_3_1_4,D1_3_1_5,D1_3_1_6,D1_3_1_7,D1_3_1_8,DATE
0,41804,3171025621,3171025600,울산광역시,울주군,온양읍,1,1960,61,0,...,1,,,,,,,,,1021
1,27700,3120011800,3120054000,울산광역시,북구,강동동,2,1990,31,0,...,1,,,,,,,,,1021
2,273273,4157010200,4157051500,경기도,김포시,김포본동,2,1976,45,0,...,1,,,,,,,,,1028
3,258483,4413110900,4413158000,충청남도,천안시 동남구,청룡동,1,1955,66,0,...,2,4,,1,4413110900,4413158000,충청남도,천안시 동남구,청룡동,1028
4,267738,4111313100,4111356000,경기도,수원시 권선구,서둔동,2,1983,38,0,...,2,4,,1,4111313100,4111356000,경기도,수원시 권선구,서둔동,1028


In [93]:
from pathlib import Path
import pandas as pd
import duckdb

excelfile_sheets = {
    "../data/raw/개인통행실태조사(2021년기준)/Code book.xlsx": [
        ("기본data", 1),
        ("이동data", 1),
        ("Value", 1),
    ],
    "../data/raw/개인통행실태조사(2021년기준)/행정동코드_20210726(말소코드포함).xlsx": [
        ("행정동", 0)
    ],
}


# Connect to DuckDB
con = duckdb.connect(database="../data/개인통행실태조사.db")

# Loop through the files and sheets
for file_path, sheets in excelfile_sheets.items():
    for sheet_name, skiprows in sheets:
        # Read the sheet into a DataFrame, skipping the specified rows
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skiprows)

        # Create a unique table name
        table_name = f"{sheet_name}_{Path(file_path).stem}"
        table_name = table_name.replace(" ", "_").replace("(", "_").replace(")", "_")
        print(table_name)

        # Load the DataFrame into DuckDB, forcefully
        con.sql(f"DROP TABLE IF EXISTS {table_name}")
        con.sql(f"CREATE TABLE {table_name} AS SELECT * FROM df")

# Example: Query a table in DuckDB
result = con.sql("SELECT * FROM 이동data_Code_book").fetchmany(5)
print(result)

# Close the DuckDB connection when done
con.close()


기본data_Code_book
이동data_Code_book
Value_Code_book
행정동_행정동코드_20210726_말소코드포함_
[('idx', '조사ID'), ('fid', '통행 고유번호 '), ('th_seq', '통행 수'), ('sTP1', '출발지'), ('sTP1_1_4', '출발지 장소 - 법정동코드')]


In [116]:
import duckdb
import pandas as pd

con = duckdb.connect("../data/개인통행실태조사.db")


# 모든 테이블을 가져오는 쿼리
tables_query = (
    "SELECT table_name FROM information_schema.tables WHERE table_schema='main'"
)

# 모든 테이블 조회
tables = con.sql(tables_query).fetchall()


def show_columns(table_name):
    print(f"Table: {table_name}")

    # 해당 테이블의 컬럼을 가져오는 쿼리
    columns_query = f"""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = '{table_name}'
    """

    # 컬럼 정보 출력
    columns = con.sql(columns_query).fetchall()
    for column in columns:
        column_name, data_type = column
        print(f"    Column: {column_name}, Type: {data_type}")


# 각 테이블의 컬럼 정보를 나열
for table in tables:
    show_columns(table[0])


# Close the DuckDB connection when done
con.close()

Table: Value_Code_book
    Column: 값, Type: VARCHAR
    Column: Unnamed: 1, Type: VARCHAR
    Column: 레이블, Type: VARCHAR
Table: ①개인특성
    Column: idx, Type: BIGINT
    Column: SQ1_4, Type: BIGINT
    Column: SQ1_5, Type: BIGINT
    Column: SQ1_6, Type: VARCHAR
    Column: SQ1_7, Type: VARCHAR
    Column: SQ1_8, Type: VARCHAR
    Column: SQ2, Type: BIGINT
    Column: SQ3, Type: BIGINT
    Column: SQ3_1, Type: BIGINT
    Column: Q1_1, Type: VARCHAR
    Column: Q1_2, Type: VARCHAR
    Column: Q1_5, Type: VARCHAR
    Column: Q1_3, Type: VARCHAR
    Column: Q1_4, Type: VARCHAR
    Column: Q1, Type: VARCHAR
    Column: Q2, Type: VARCHAR
    Column: Q2_a, Type: VARCHAR
    Column: Q3, Type: VARCHAR
    Column: Q4_1, Type: VARCHAR
    Column: Q4_2, Type: VARCHAR
    Column: Q4_3, Type: VARCHAR
    Column: Q4_4, Type: VARCHAR
    Column: Q4_5, Type: VARCHAR
    Column: Q4_6, Type: VARCHAR
    Column: Q4_7, Type: VARCHAR
    Column: Q4_8, Type: VARCHAR
    Column: Q4_9, Type: VARCHAR
    Column:

In [95]:
import duckdb

# DuckDB 연결
con = duckdb.connect("../data/개인통행실태조사.db")

# 테이블 목록
tables = ["①개인특성", "②이동특성"]

# 각 테이블의 레코드 수 계산
for table in tables:
    # 레코드 수 계산 쿼리
    record_count_query = f"SELECT COUNT(*) FROM {table}"
    record_count = con.sql(record_count_query).fetchone()[0]

    print(f"Table: {table}, Record Count: {record_count}")

# Close the DuckDB connection when done
con.close()

Table: ①개인특성, Record Count: 133326
Table: ②이동특성, Record Count: 356899


In [96]:
import duckdb

# DuckDB 연결
con = duckdb.connect("../data/개인통행실태조사.db")

# 테이블과 컬럼 정보
table_columns = {"①개인특성": ["idx"], "②이동특성": ["idx", "fid"]}

# 각 테이블의 컬럼별로 고유 값 수와 가장 많이 등장하는 값의 빈도를 계산
for table, columns in table_columns.items():
    print(f"Table: {table}")

    for column in columns:
        # 고유한 값의 수 계산 쿼리
        unique_count_query = f"SELECT COUNT(DISTINCT {column}) FROM {table}"
        unique_count = con.sql(unique_count_query).fetchone()[0]

        # 가장 많이 등장한 값의 빈도 계산 쿼리
        most_common_value_query = f"""
        SELECT {column}, COUNT({column}) AS cnt
        FROM {table}
        GROUP BY {column}
        ORDER BY cnt DESC
        LIMIT 1
        """
        most_common_value = con.sql(most_common_value_query).fetchone()

        print(f"    Column: {column}")
        print(f"        Unique values: {unique_count}")
        print(
            f"        Most common value: {most_common_value[0]} (Count: {most_common_value[1]})"
        )

# Close the DuckDB connection when done
con.close()

Table: ①개인특성
    Column: idx
        Unique values: 133326
        Most common value: 299932 (Count: 1)
Table: ②이동특성
    Column: idx
        Unique values: 133326
        Most common value: 254070 (Count: 12)
    Column: fid
        Unique values: 334067
        Most common value:   (Count: 22833)


In [97]:
import duckdb

# Connect to DuckDB
con = duckdb.connect(database="../data/개인통행실태조사.db")

# Example: Query a table in DuckDB
result1 = con.sql("SELECT * FROM ①개인특성 USING SAMPLE 5").df()
result2 = con.sql("SELECT * FROM ②이동특성 USING SAMPLE 5").df()
with pd.option_context("display.max_rows", 100, "display.max_columns", 100):
    display(result1, result2)

# Close the DuckDB connection when done
con.close()

,idx,SQ1_4,SQ1_5,SQ1_6,SQ1_7,SQ1_8,SQ2,SQ3,SQ3_1,Q1_1,Q1_2,Q1_5,Q1_3,Q1_4,Q1,Q2,Q2_a,Q3,Q4_1,Q4_2,Q4_3,Q4_4,Q4_5,Q4_6,Q4_7,Q4_8,Q4_9,Q4_10,Q4_97,Q4_a,Q4_1_1,Q4_1_2,Q4_1_3,Q4_1_4,Q4_1_5,Q4_1_6,Q4_1_97,R1,R1_1,R2,R2_0_1,R2_1_4,R2_1_5,R2_1_6,R2_1_7,R2_1_8,R3,R3_1_4,R3_1_5,R3_1_6,R3_1_7,R3_1_8,R3_2,R3_2_a,R3_3,R3_3_a,R3_4,R3_4_a,R3_5,R4_1,R4_2,R4_3,R4_4,R4_5_1_1,R4_5_1_2,R4_5_2_1,R4_5_2_2,R4_6_1_1,R4_6_1_2,R4_6_2_1,R4_6_2_2,R5,D1,D1_2,D1_2_a,D1_3,D1_3_1_4,D1_3_1_5,D1_3_1_6,D1_3_1_7,D1_3_1_8,DATE
0,36953,1138010700,1138058000,서울특별시,은평구,응암제1동,1,1951,70,0,0,0,0,1,1,3,,1,0,0,0,0,0,0,0,0,0,0,0,,,,,,,,,2,,99,,,,,,,2,,,,,,3,,4,,98,,98,,,,,,,,,,,,,1,1,,,,,,,,,1021
1,278668,1111018700,1111057000,서울특별시,종로구,무악동,2,1966,55,0,0,0,4,0,4,1,,4,8,0,0,0,0,0,0,0,0,0,0,,0,,,,,,,1,,99,,,,,,,1,1141011700,1141061500,서울특별시,서대문구,연희동,6,,2,,98,,98,,,2,,8,10,20,30,,,,,1,1,,,,,,,,,1028
2,289397,4311210400,4311255000,충청북도,청주시 서원구,산남동,2,1990,31,0,0,0,4,0,4,1,,4,3,0,0,0,0,0,0,0,0,0,0,,0,,,,,,,1,,99,,,,,,,1,4311112800,4311173000,충청북도,청주시 상당구,용암2동,2,,2,,2,,98,1,,2,,6,20,16,5,,,,,2,1,,,,,,,,,1104
3,244763,1168011200,1168070000,서울특별시,강남구,세곡동,2,1968,53,0,1,0,2,0,3,1,,3,1,0,0,0,0,0,0,0,0,0,0,,0,,,,,,,1,,99,,,,,,,2,,,,,,5,,4,,4,,98,98,,,,,,,,,,,,1,1,,,,,,,,,1028
4,281651,4211011800,4211070500,강원도,춘천시,신사우동,2,1967,54,0,0,0,3,0,3,1,,2,2,0,0,0,0,0,0,0,0,0,0,,0,,,,,,,1,,99,,,,,,,1,4372036029,4372036000,충청북도,보은군,수한면,5,,2,,98,,98,,,2,,8,0,18,0,,,,,1,2,97,,1,4211011800,4211070500,강원도,춘천시,신사우동,1028


,idx,fid,th_seq,sTP1,sTP1_1_4,sTP1_1_5,sTP1_1_6,sTP1_1_7,sTP1_1_8,TP1,TP1_1_4,TP1_1_5,TP1_1_6,TP1_1_7,TP1_1_8,TP2,TP2_a,TP3_1,TP3_2,TP4_1,TP4_2,TP5_1,TP5_1_a,TP5_1_1_4,TP5_1_1_5,TP5_1_1_6,TP5_1_1_7,TP5_1_1_8,TP5_1_2_1,TP5_1_2_2,TP5_1_2_3,TP5_1_2_4,TP5_1_10,TP5_1_11_1,TP5_1_11_2,TP5_1_11_3,TP5_1_11_4,TP5_1_12_1,TP5_1_12_2,TP5_1_13,TP5_1_14,TP5_1_t1,TP5_2,TP5_2_a,TP5_2_1_4,TP5_2_1_5,TP5_2_1_6,TP5_2_1_7,TP5_2_1_8,TP5_2_2_1,...,TP5_8_11_2,TP5_8_11_3,TP5_8_11_4,TP5_8_12_1,TP5_8_12_2,TP5_8_13,TP5_8_14,TP5_8_t1,TP5_9,TP5_9_a,TP5_9_1_4,TP5_9_1_5,TP5_9_1_6,TP5_9_1_7,TP5_9_1_8,TP5_9_2_1,TP5_9_2_2,TP5_9_2_3,TP5_9_2_4,TP5_9_10,TP5_9_11_1,TP5_9_11_2,TP5_9_11_3,TP5_9_11_4,TP5_9_12_1,TP5_9_12_2,TP5_9_13,TP5_9_14,TP5_9_t1,TP5_10,TP5_10_a,TP5_10_1_4,TP5_10_1_5,TP5_10_1_6,TP5_10_1_7,TP5_10_1_8,TP5_10_2_1,TP5_10_2_2,TP5_10_2_3,TP5_10_2_4,TP5_10_10,TP5_10_11_1,TP5_10_11_2,TP5_10_11_3,TP5_10_11_4,TP5_10_12_1,TP5_10_12_2,TP5_10_13,TP5_10_14,TP5_10_t1
0,18156,23477,5,97,3611011200,3611055000,세종특별자치시,,고운동,1,3611011200,3611055000,세종특별자치시,,고운동,1,,11,10,11,25,1,,3611011200,3611055000,세종특별자치시,,고운동,,,,,,,,,,,,,,4,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
1,308378,10332493,2,97,4121010200,4121061000,경기도,광명시,철산3동,1,4121010200,4121059000,경기도,광명시,철산1동,1,,10,15,10,30,2,,4121010200,4121059000,경기도,광명시,철산1동,1,1,2,1,,,,,,,,,,4,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
2,264603,10197347,2,3,1129013900,1129081000,서울특별시,성북구,석관동,1,1129013900,1129081000,서울특별시,성북구,석관동,1,,13,0,13,10,1,,1129013900,1129081000,서울특별시,성북구,석관동,,,,,,,,,,,,,,3,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
3,280570,10248989,1,1,4684025622,4684025600,전라남도,무안군,삼향읍,97,4689025026,4689025000,전라남도,완도군,완도읍,6,,6,0,7,30,2,,4689025026,4689025000,전라남도,완도군,완도읍,1,2,2,1,,,,,,,,,,16,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
4,279562,120021422,1,1,4374032021,4374032000,충청북도,영동군,황간면,97,1168010400,1168056500,서울특별시,강남구,청담동,11,,7,0,10,45,2,,1168010400,1168056500,서울특별시,강남구,청담동,2,2,3,1,,,,,,,,,,29,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,


In [98]:
import duckdb

# DuckDB 파일 기반 데이터베이스 연결
con = duckdb.connect("../data/개인통행실태조사.db")

# DATE 컬럼의 모든 고유한 값 확인 쿼리
unique_dates_query = "SELECT DISTINCT DATE FROM ①개인특성 ORDER BY DATE"

# 고유한 값 조회 및 출력
unique_dates = con.sql(unique_dates_query).fetchall()
print("Unique DATE values:", [date[0] for date in unique_dates])

# 연결 닫기
con.close()


Unique DATE values: [1021, 1028, 1104, 1111, 1117, 1118, 1123, 1124, 1125, 1130, 1201, 1202, 1207, 1208, 1209, 1215]


In [99]:
import duckdb
import pandas as pd

# DuckDB 파일 기반 데이터베이스 연결
con = duckdb.connect("../data/개인통행실태조사.db")

# 1. idx가 284968인 데이터를 찾고 DataFrame으로 변환
idx_query = "SELECT idx, fid, th_seq FROM ②이동특성 WHERE idx = 284968"
idx_df = con.sql(idx_query).df()

# 2. fid가 10264014인 데이터를 찾고 DataFrame으로 변환
fid_query = "SELECT * FROM ②이동특성 WHERE fid = '10264014'"
fid_df = con.sql(fid_query).df()

# pandas 출력 옵션을 설정하여 더 많은 행과 열 출력
with pd.option_context("display.max_rows", 100, "display.max_columns", 100):
    display(idx_df, fid_df)

# 연결 닫기
con.close()


,idx,fid,th_seq
0,284968,10262912,1
1,284968,10263460,2
2,284968,10263737,3
3,284968,10263862,4
4,284968,10264014,5
5,284968,10264140,6
6,284968,10264170,7
7,284968,10264339,8


,idx,fid,th_seq,sTP1,sTP1_1_4,sTP1_1_5,sTP1_1_6,sTP1_1_7,sTP1_1_8,TP1,TP1_1_4,TP1_1_5,TP1_1_6,TP1_1_7,TP1_1_8,TP2,TP2_a,TP3_1,TP3_2,TP4_1,TP4_2,TP5_1,TP5_1_a,TP5_1_1_4,TP5_1_1_5,TP5_1_1_6,TP5_1_1_7,TP5_1_1_8,TP5_1_2_1,TP5_1_2_2,TP5_1_2_3,TP5_1_2_4,TP5_1_10,TP5_1_11_1,TP5_1_11_2,TP5_1_11_3,TP5_1_11_4,TP5_1_12_1,TP5_1_12_2,TP5_1_13,TP5_1_14,TP5_1_t1,TP5_2,TP5_2_a,TP5_2_1_4,TP5_2_1_5,TP5_2_1_6,TP5_2_1_7,TP5_2_1_8,TP5_2_2_1,...,TP5_8_11_2,TP5_8_11_3,TP5_8_11_4,TP5_8_12_1,TP5_8_12_2,TP5_8_13,TP5_8_14,TP5_8_t1,TP5_9,TP5_9_a,TP5_9_1_4,TP5_9_1_5,TP5_9_1_6,TP5_9_1_7,TP5_9_1_8,TP5_9_2_1,TP5_9_2_2,TP5_9_2_3,TP5_9_2_4,TP5_9_10,TP5_9_11_1,TP5_9_11_2,TP5_9_11_3,TP5_9_11_4,TP5_9_12_1,TP5_9_12_2,TP5_9_13,TP5_9_14,TP5_9_t1,TP5_10,TP5_10_a,TP5_10_1_4,TP5_10_1_5,TP5_10_1_6,TP5_10_1_7,TP5_10_1_8,TP5_10_2_1,TP5_10_2_2,TP5_10_2_3,TP5_10_2_4,TP5_10_10,TP5_10_11_1,TP5_10_11_2,TP5_10_11_3,TP5_10_11_4,TP5_10_12_1,TP5_10_12_2,TP5_10_13,TP5_10_14,TP5_10_t1
0,284968,10264014,5,1,4311112100,4311169000,충청북도,청주시 상당구,용담.명암.산성동,97,4311111200,4311152500,충청북도,청주시 상당구,중앙동,9,,16,30,16,50,2,,4311111200,4311152500,충청북도,청주시 상당구,중앙동,1,1,4,1,,,,,,,,,,5,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,


In [120]:
import duckdb

# DuckDB 파일 기반 데이터베이스 연결
con = duckdb.connect("../data/개인통행실태조사.db")

# SQL 쿼리: Q1_1가 1보다 크고 D1이 1인 경우 idx를 추출
# 가구원에 만 5세 미만 아동이 있고, 조사 당일 통행한 경우
query = """
SELECT idx
FROM ①개인특성
WHERE TRY_CAST(Q1_1 AS BIGINT) > 0
    AND D1 = 1;
"""

# Constructing Relations
# Relations can be created from SQL queries using the duckdb.sql method.
result = con.sql(query)

# 결과 출력
print(result.df())

# ②이동특성 테이블과 조인하여 idx를 필터링
query_join = """
SELECT t2.*
FROM ②이동특성 AS t2
JOIN result AS t1
ON t2.idx = t1.idx;
"""

# 쿼리 실행 및 결과 DataFrame으로 변환
filtered_result = con.sql(query_join).df()

# pandas 출력 옵션을 설정하여 더 많은 행과 열 출력
with pd.option_context("display.max_rows", 100, "display.max_columns", 100):
    display(filtered_result)

# 연결 닫기
con.close()


         idx
0      10099
1      10129
2      10149
3      10209
4      10232
...      ...
8133  338487
8134  339031
8135  339119
8136  339207
8137  340342

[8138 rows x 1 columns]


,idx,fid,th_seq,sTP1,sTP1_1_4,sTP1_1_5,sTP1_1_6,sTP1_1_7,sTP1_1_8,TP1,TP1_1_4,TP1_1_5,TP1_1_6,TP1_1_7,TP1_1_8,TP2,TP2_a,TP3_1,TP3_2,TP4_1,TP4_2,TP5_1,TP5_1_a,TP5_1_1_4,TP5_1_1_5,TP5_1_1_6,TP5_1_1_7,TP5_1_1_8,TP5_1_2_1,TP5_1_2_2,TP5_1_2_3,TP5_1_2_4,TP5_1_10,TP5_1_11_1,TP5_1_11_2,TP5_1_11_3,TP5_1_11_4,TP5_1_12_1,TP5_1_12_2,TP5_1_13,TP5_1_14,TP5_1_t1,TP5_2,TP5_2_a,TP5_2_1_4,TP5_2_1_5,TP5_2_1_6,TP5_2_1_7,TP5_2_1_8,TP5_2_2_1,...,TP5_8_11_2,TP5_8_11_3,TP5_8_11_4,TP5_8_12_1,TP5_8_12_2,TP5_8_13,TP5_8_14,TP5_8_t1,TP5_9,TP5_9_a,TP5_9_1_4,TP5_9_1_5,TP5_9_1_6,TP5_9_1_7,TP5_9_1_8,TP5_9_2_1,TP5_9_2_2,TP5_9_2_3,TP5_9_2_4,TP5_9_10,TP5_9_11_1,TP5_9_11_2,TP5_9_11_3,TP5_9_11_4,TP5_9_12_1,TP5_9_12_2,TP5_9_13,TP5_9_14,TP5_9_t1,TP5_10,TP5_10_a,TP5_10_1_4,TP5_10_1_5,TP5_10_1_6,TP5_10_1_7,TP5_10_1_8,TP5_10_2_1,TP5_10_2_2,TP5_10_2_3,TP5_10_2_4,TP5_10_10,TP5_10_11_1,TP5_10_11_2,TP5_10_11_3,TP5_10_11_4,TP5_10_12_1,TP5_10_12_2,TP5_10_13,TP5_10_14,TP5_10_t1
0,75177,206207,1,1,4511312200,4511364100,전라북도,전주시 덕진구,송천1동,97,4518011900,4518053500,전라북도,정읍시,내장상동,8,,11,0,12,0,2,,4518011900,4518053500,전라북도,정읍시,내장상동,2,2,1,1,,,,,,,,,,13,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
1,75177,206235,2,97,4518011900,4518053500,전라북도,정읍시,내장상동,1,4511312200,4511364100,전라북도,전주시 덕진구,송천1동,1,,15,0,16,0,2,,4511312200,4511364100,전라북도,전주시 덕진구,송천1동,2,2,1,1,,,,,,,,,,13,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
2,75184,206184,2,97,4280025021,4280025000,강원도,양구군,양구읍,97,4280025021,4280025000,강원도,양구군,양구읍,8,,9,30,10,0,1,,4280025021,4280025000,강원도,양구군,양구읍,,,,,,,,,,,,,,7,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
3,75184,206250,1,1,4280025021,4280025000,강원도,양구군,양구읍,97,4280025021,4280025000,강원도,양구군,양구읍,12,,9,20,9,25,1,,4280025021,4280025000,강원도,양구군,양구읍,,,,,,,,,,,,,,2,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
4,75184,206287,3,97,4280025021,4280025000,강원도,양구군,양구읍,1,4280025021,4280025000,강원도,양구군,양구읍,1,,10,20,10,50,1,,4280025021,4280025000,강원도,양구군,양구읍,,,,,,,,,,,,,,7,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26630,75118,206121,5,1,4311210600,4311256000,충청북도,청주시 서원구,분평동,97,4311210600,4311256000,충청북도,청주시 서원구,분평동,12,,16,35,16,45,1,,4311210600,4311256000,충청북도,청주시 서원구,분평동,,,,,,,,,,,,,,3,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
26631,75118,206140,6,97,4311210600,4311256000,충청북도,청주시 서원구,분평동,97,4311210600,4311256000,충청북도,청주시 서원구,분평동,7,,16,50,17,0,1,,4311210600,4311256000,충청북도,청주시 서원구,분평동,,,,,,,,,,,,,,3,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
26632,75118,206237,7,97,4311210600,4311256000,충청북도,청주시 서원구,분평동,1,4311210600,4311256000,충청북도,청주시 서원구,분평동,1,,17,15,17,20,1,,4311210600,4311256000,충청북도,청주시 서원구,분평동,,,,,,,,,,,,,,2,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,
26633,75130,206183,1,1,4873025325,4873025300,경상남도,함안군,칠원읍,2,4873025321,4873025300,경상남도,함안군,칠원읍,3,,7,20,7,30,2,,4873025321,4873025300,경상남도,함안군,칠원읍,1,1,4,1,,,,,,,,,,3,,,,,None,None,None,,...,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,,,,,,None,None,None,,,,,,,,,,,,,,


In [129]:
for c in filtered_result.columns.to_list():
    print(c, ",", sep="")

idx,
fid,
th_seq,
sTP1,
sTP1_1_4,
sTP1_1_5,
sTP1_1_6,
sTP1_1_7,
sTP1_1_8,
TP1,
TP1_1_4,
TP1_1_5,
TP1_1_6,
TP1_1_7,
TP1_1_8,
TP2,
TP2_a,
TP3_1,
TP3_2,
TP4_1,
TP4_2,
TP5_1,
TP5_1_a,
TP5_1_1_4,
TP5_1_1_5,
TP5_1_1_6,
TP5_1_1_7,
TP5_1_1_8,
TP5_1_2_1,
TP5_1_2_2,
TP5_1_2_3,
TP5_1_2_4,
TP5_1_10,
TP5_1_11_1,
TP5_1_11_2,
TP5_1_11_3,
TP5_1_11_4,
TP5_1_12_1,
TP5_1_12_2,
TP5_1_13,
TP5_1_14,
TP5_1_t1,
TP5_2,
TP5_2_a,
TP5_2_1_4,
TP5_2_1_5,
TP5_2_1_6,
TP5_2_1_7,
TP5_2_1_8,
TP5_2_2_1,
TP5_2_2_2,
TP5_2_2_3,
TP5_2_2_4,
TP5_2_10,
TP5_2_11_1,
TP5_2_11_2,
TP5_2_11_3,
TP5_2_11_4,
TP5_2_12_1,
TP5_2_12_2,
TP5_2_13,
TP5_2_14,
TP5_2_t1,
TP5_3,
TP5_3_a,
TP5_3_1_4,
TP5_3_1_5,
TP5_3_1_6,
TP5_3_1_7,
TP5_3_1_8,
TP5_3_2_1,
TP5_3_2_2,
TP5_3_2_3,
TP5_3_2_4,
TP5_3_10,
TP5_3_11_1,
TP5_3_11_2,
TP5_3_11_3,
TP5_3_11_4,
TP5_3_12_1,
TP5_3_12_2,
TP5_3_13,
TP5_3_14,
TP5_3_t1,
TP5_4,
TP5_4_a,
TP5_4_1_4,
TP5_4_1_5,
TP5_4_1_6,
TP5_4_1_7,
TP5_4_1_8,
TP5_4_2_1,
TP5_4_2_2,
TP5_4_2_3,
TP5_4_2_4,
TP5_4_10,
TP5_4_11_1,
TP5_4_11_2,
TP5_4_

In [ ]:
import duckdb

# DuckDB 파일 기반 데이터베이스 연결
con = duckdb.connect("../data/개인통행실태조사.db")

# SQL 쿼리: Q1_1가 1보다 크고 D1이 1인 경우 idx를 추출
# 가구원에 만 5세 미만 아동이 있고, 조사 당일 통행한 경우
query = """
SELECT idx
FROM ①개인특성
WHERE TRY_CAST(Q1_1 AS BIGINT) > 0
    AND D1 = 1;
"""

# Constructing Relations
# Relations can be created from SQL queries using the duckdb.sql method.
result = con.sql(query)

# ②이동특성 테이블과 조인하여 idx를 필터링
query_join = """
SELECT t2.*
FROM ②이동특성 AS t2
JOIN result AS t1
ON t2.idx = t1.idx;
"""

# 쿼리 실행
filtered_result = con.sql(query_join)

# TP5_1_2_2의 고유값 조회 쿼리 작성
unique_values_query = """
SELECT DISTINCT TP5_1_2_2
FROM filtered_result;
"""

# 쿼리 실행 및 결과 DataFrame으로 변환
unique_values_df = con.sql(unique_values_query).df()

# 고유값 출력
display(unique_values_df)

# 연결 닫기
con.close()


,TP5_1_2_2
0,4
1,9
2,3
3,2
4,1
5,11
6,7
7,15
8,10
9,8


In [147]:
import duckdb

# DuckDB 파일 기반 데이터베이스 연결
con = duckdb.connect("../data/개인통행실태조사.db")

# SQL 쿼리: Q1_1가 1보다 크고 D1이 1인 경우 idx를 추출
# 가구원에 만 5세 미만 아동이 있고, 조사 당일 통행한 경우
query = """
SELECT idx
FROM ①개인특성
WHERE TRY_CAST(Q1_1 AS BIGINT) > 0
    AND D1 = 1;
"""

# Constructing Relations
# Relations can be created from SQL queries using the duckdb.sql method.
result = con.sql(query)

# ②이동특성 테이블과 조인하여 idx를 필터링
query_join = """
SELECT t2.*
FROM ②이동특성 AS t2
JOIN result AS t1
ON t2.idx = t1.idx;
"""

# 쿼리 실행
filtered_result = con.sql(query_join)

# TP5_1	1번째 이동 수단 승용차/승합차 TP5_1 = '2'
# TP5_1_2_2	1번째 이동 수단 _ (승용차/승합차) 운전자 포함 탑승 인원: 2인 이상
column_filter = """
SELECT 
    idx,
    fid,
    th_seq,
    TP1,
    TP2
FROM filtered_result
WHERE TP5_1 = '2'
    AND TRY_CAST(TP5_1_2_2 AS BIGINT) >= 2;
"""

# 쿼리 실행
column_result = con.sql(column_filter)
print(column_result)

column_result_df = column_result.df()
# column_result_df.to_csv(
#     "../results/개인통행실태조사_2021_목요일_만5세미만가구_승용차승합차_탑승2인이상.csv",
#     index=False,
#     encoding="utf-8-sig",
# )
column_result_df.to_excel(
    "../results/개인통행실태조사_2021_목요일_만5세미만가구_승용차승합차_탑승2인이상.xlsx",
    index=False,
)

# 연결 닫기
con.close()


┌────────┬───────────┬────────┬─────────┬─────────┐
│  idx   │    fid    │ th_seq │   TP1   │   TP2   │
│ int64  │  varchar  │ int64  │ varchar │ varchar │
├────────┼───────────┼────────┼─────────┼─────────┤
│  12558 │ 20030529  │      2 │ 97      │ 7       │
│  12558 │ 20030530  │      3 │ 1       │ 1       │
│  12607 │ 4384      │      1 │ 97      │ 11      │
│  12607 │ 5554      │      6 │ 97      │ 11      │
│  12683 │ 4829      │      1 │ 97      │ 12      │
│  12683 │ 5427      │      4 │ 1       │ 1       │
│  12705 │ 90364     │      1 │ 97      │ 12      │
│  12714 │ 100005181 │      1 │ 97      │ 12      │
│  12714 │ 100005182 │      2 │ 97      │ 9       │
│  12714 │ 100005183 │      3 │ 97      │ 7       │
│    ·   │    ·      │      · │ ·       │ ·       │
│    ·   │    ·      │      · │ ·       │ ·       │
│    ·   │    ·      │      · │ ·       │ ·       │
│ 338178 │ 10377280  │      7 │ 1       │ 1       │
│ 338321 │ 10377501  │      1 │ 97      │ 11      │
│ 338321 │ 1